# Chapter 7 (b) – Effective Agentic System Design Techniques _(continued)_
---

Install dependencies

In [ ]:
!pip install crewai langchain-openai

In [ ]:
import getpass
import os

api_key = getpass.getpass(prompt="Enter OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown, HTML

llm = ChatOpenAI(model="gpt-4o")